In [1]:
import numpy as np
import keras
from keras.utils import np_utils
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten
from keras.layers import BatchNormalization,Activation
import keras.backend as K
import tensorflow as tf
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
model = keras.applications.DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3),pooling='avg')

In [3]:
# for layers in model.layers[:100]:
#     layers.trainable=False

In [4]:
top_model=Sequential()
top_model.add(Dense(100))
top_model.add(Activation('softmax'))
model =Model(inputs=model.input,outputs=top_model(model.output))
model.load_weights('weights.02-0.86.hdf5')

In [5]:
adam=keras.optimizers.Adam(lr =0.0001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5'
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
mcCallBack = keras.callbacks.ModelCheckpoint(filepath,save_best_only=True,mode='auto')
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20)
decaylearn = keras.callbacks.ReduceLROnPlateau(patience=10)

In [7]:
train_dir='cifar100_train'
valid_dir='cifar100_valid'

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    vertical_flip=False,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=32,target_size=(224,224),class_mode='categorical')
test_generator=test_datagen.flow_from_directory(valid_dir,batch_size=32,target_size=(224,224),class_mode='categorical')

Found 47500 images belonging to 100 classes.
Found 2500 images belonging to 100 classes.


In [9]:
model.fit_generator(train_generator,
                    epochs=25,
                    callbacks=[tbCallBack,mcCallBack,earlystop,decaylearn],
                    validation_data=test_generator)

Epoch 1/25
1485/1485 [==============================] - 1336s 900ms/step - loss: 0.1812 - acc: 0.9431 - val_loss: 0.8691 - val_acc: 0.7856
Epoch 2/25
1485/1485 [==============================] - 1308s 881ms/step - loss: 0.1559 - acc: 0.9530 - val_loss: 0.8568 - val_acc: 0.7820
Epoch 3/25
1485/1485 [==============================] - 1310s 882ms/step - loss: 0.1365 - acc: 0.9585 - val_loss: 0.8773 - val_acc: 0.7828
Epoch 4/25
1485/1485 [==============================] - 1309s 881ms/step - loss: 0.1248 - acc: 0.9614 - val_loss: 0.8826 - val_acc: 0.7800
Epoch 5/25
1144/1485 [======================>.......] - ETA: 4:55 - loss: 0.1087 - acc: 0.9679

KeyboardInterrupt: 